In [1]:
pip install Keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:

import re
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score


In [5]:
df=pd.read_csv('train.csv.zip')
df.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [6]:
df.info

<bound method DataFrame.info of        Labels                                               Text  \
0           1  Says the Annies List political group supports ...   
1           2  When did the decline of coal start? It started...   
2           3  Hillary Clinton agrees with John McCain "by vo...   
3           1  Health care reform legislation is likely to ma...   
4           2  The economic turnaround started at the end of ...   
...       ...                                                ...   
10235       3  There are a larger number of shark attacks in ...   
10236       3  Democrats have now become the party of the [At...   
10237       2  Says an alternative to Social Security that op...   
10238       1  On lifting the U.S. Cuban embargo and allowing...   
10239       4  The Department of Veterans Affairs has a manua...   

                                 Text_Tag  
0                                abortion  
1      energy,history,job-accomplishments  
2                  

In [7]:
df.describe()

,Labels
count,10240.000000
mean,2.328613
std,1.650933
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,5.000000


In [8]:
df[df.duplicated(['Text'])]

,Labels,Text,Text_Tag
1014,2,On abortion,"abortion,candidates-biography"
1814,1,On support for gay marriage.,"civil-rights,families,gays-and-lesbians,marriage"
1846,1,"Obama says Iran is a 'tiny' country, 'doesn't ...",foreign-policy
2697,1,On repealing the 17th Amendment,"debates,elections,states"
2846,3,"Four balanced budgets in a row, with no new ta...","job-accomplishments,jobs,state-budget,state-fi..."
3256,1,On a cap-and-trade plan.,"cap-and-trade,climate-change,environment"
4386,1,On the Trans-Pacific Partnership.,trade
4839,2,During Sherrod Browns past decade as a D.C. po...,"economy,job-accomplishments,jobs"
4940,1,On changing the rules for filibusters on presi...,congressional-rules
6759,2,On torture.,"human-rights,terrorism"


In [9]:
df[df.duplicated(['Text_Tag'])]

,Labels,Text,Text_Tag
14,0,Most of the (Affordable Care Act) has already ...,health-care
15,2,"In this last election in November, ... 63 perc...",elections
17,0,"U.S. Rep. Ron Kind, D-Wis., and his fellow Dem...",federal-budget
25,1,I dont know who (Jonathan Gruber) is.,health-care
27,2,Rick Perry has never lost an election and rema...,candidates-biography
...,...,...,...
10228,0,Stopped by Smiley Cookie to pick up some great...,food
10231,2,"When it comes to the state deficit, Wisconsin ...",state-budget
10232,2,Eighty percent of the net new jobs created in ...,"economy,jobs"
10236,3,Democrats have now become the party of the [At...,elections


In [10]:
df=df.drop_duplicates(['Text'])

In [11]:
df=df.drop_duplicates(['Text_Tag'])

In [12]:
df.sample(3)

,Labels,Text,Text_Tag
5043,2,"Says Anthropology is a STEM (science, technolo...","education,science"
3909,2,Although we have twice the population of Greec...,"economy,state-finances"
4372,1,This Congressadjourned earliest of any time in...,"congress,elections,history"


In [13]:
df.isnull().sum()

Labels      0
Text        0
Text_Tag    1
dtype: int64

In [14]:
df.dropna(inplace=True)

-----------------------------------------------------------------------------

In [15]:
import string
regular_punct = list(string.punctuation)
def remove_punctuation(text,punct_list):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()


convertig into lower

In [16]:
df["Text"] = df["Text"].apply(lambda x:x.lower())

In [17]:
df["Text"][0]

'says the annies list political group supports third-trimester abortions on demand.'

In [18]:
df['Text_Tag']=df['Text_Tag'].apply(lambda x:x.lower())

In [19]:
df['Text_Tag'][0]

'abortion'

-----------------------------------------------------------------------------------------------------------------------------

In [20]:
import string
special = string.punctuation
special

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
df["Text"] = df["Text"].apply(lambda x:re.sub("[#!$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]","",x))

In [22]:
df['Text_Tag']=df['Text_Tag'].apply(lambda x:re.sub('[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',"",x))

----------------------------------------------------------------------------------------------------------------------------

Removing Stop Words

In [23]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\javee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
stope = stopwords.words('english')
stope

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [25]:
df["Text"][0]

'says the annies list political group supports thirdtrimester abortions on demand'

In [26]:
df["Text"] = df["Text"].apply(lambda x:" ".join(i for i in x.split(" ") if i not in stope))

In [27]:
df['Text'][0]

'says annies list political group supports thirdtrimester abortions demand'

--------------------------------------------------------------------------------------------------------------------------------

In [28]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\javee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\javee\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [29]:
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

In [30]:
df['Text']=df['Text'].apply(lambda x:"".join (lemmatize_words(x)))

In [31]:
df['Text'][0]

'says annies list political group supports thirdtrimester abortions demand'

In [32]:
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]


In [33]:
df['Text_Tag']=df['Text_Tag'].apply(lambda x:"".join (lemmatize_words(x)))

In [34]:
df['Text_Tag'][0]

'abortion'

-------------------------------------------------------------------------------------------------------------------------------

In [35]:
df

,Labels,Text,Text_Tag
0,1,says annies list political group supports thir...,abortion
1,2,decline coal start started natural gas took st...,energyhistoryjobaccomplishments
2,3,"hillary clinton agrees john mccain ""by voting ...",foreignpolicy
3,1,health care reform legislation likely mandate ...,healthcare
4,2,economic turnaround started end term,economyjobs
...,...,...,...
10233,4,mayor fung wants punish childrens education re...,childrencitybudgetdeficiteducationstatebudgett...
10234,2,ruling supreme court lobbyist could go legisla...,corporationselections
10235,3,larger number shark attacks florida cases vote...,animalselections
10237,2,says alternative social security operates galv...,retirementsocialsecurity


In [36]:
#Text= pd.DataFrame(df['Text'].tolist()).astype(str)
#Text_Tag = pd.DataFrame(df['Text_Tag'].tolist()).astype(str)

In [37]:
df['NewsText'] = df['Text_Tag'].astype(str) +" "+ df['Text']
df['NewsText'] = df['Text_Tag'].astype(str) +" "+ df['Text']
df

,Labels,Text,Text_Tag,NewsText
0,1,says annies list political group supports thir...,abortion,abortion says annies list political group supp...
1,2,decline coal start started natural gas took st...,energyhistoryjobaccomplishments,energyhistoryjobaccomplishments decline coal s...
2,3,"hillary clinton agrees john mccain ""by voting ...",foreignpolicy,foreignpolicy hillary clinton agrees john mcca...
3,1,health care reform legislation likely mandate ...,healthcare,healthcare health care reform legislation like...
4,2,economic turnaround started end term,economyjobs,economyjobs economic turnaround started end term
...,...,...,...,...
10233,4,mayor fung wants punish childrens education re...,childrencitybudgetdeficiteducationstatebudgett...,childrencitybudgetdeficiteducationstatebudgett...
10234,2,ruling supreme court lobbyist could go legisla...,corporationselections,corporationselections ruling supreme court lob...
10235,3,larger number shark attacks florida cases vote...,animalselections,animalselections larger number shark attacks f...
10237,2,says alternative social security operates galv...,retirementsocialsecurity,retirementsocialsecurity says alternative soci...


In [38]:
#df['Text']=df['Text'].apply(lambda x: str(x))
#df['Text_Tag']=df['Text_Tag'].apply(lambda x: str(x))

In [39]:
type(df['Text'][2])

str

In [40]:
df=df.drop(columns=['Text','Text_Tag'])

--------------------------------------------------------------------------------------------------------------------------------

In [41]:
df['column']=df['NewsText'].apply(lambda x:len(x.split(" ")))


In [42]:
df['column'].value_counts()

11     373
10     363
9      343
12     338
13     299
8      295
14     270
15     242
7      212
16     200
17     148
6      121
18     118
19     116
20      73
5       58
21      50
22      40
4       37
23      33
24      18
25      18
3       12
26      10
28       7
30       7
27       5
29       4
34       3
2        2
32       2
35       1
31       1
196      1
38       1
48       1
Name: column, dtype: int64

-----------------------------------------------------------------------------------------------------------------------------

Train Test Split

In [43]:
x=df.drop(['Labels','column'],axis=1)
y=df['Labels']

In [44]:
x_train, x_test, y_train, y_test = train_test_split(x, y,shuffle=True, test_size=0.33, random_state=111)

In [45]:
x.head()

,NewsText
0,abortion says annies list political group supp...
1,energyhistoryjobaccomplishments decline coal s...
2,foreignpolicy hillary clinton agrees john mcca...
3,healthcare health care reform legislation like...
4,economyjobs economic turnaround started end term


In [46]:
x.shape

(3822, 1)

In [47]:
y.shape

(3822,)

In [48]:
x_train.shape

(2560, 1)

In [49]:
y_train.shape

(2560,)

In [50]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [51]:
# tokenize text
token=Tokenizer()
token.fit_on_texts(x_train.NewsText)
word_index=token.word_index
vocab_size=len(word_index)

In [52]:
len(word_index)

9275

In [53]:
vocab_size

9275

--------------------------------------------------------------------------------------------------------------------------------

In [54]:
 # padding data
Sequences= token.texts_to_sequences(df['NewsText'])
padded_seq= pad_sequences(Sequences, maxlen=20,padding='post',truncating='post')

In [55]:
padded_seq[1]

array([5102,  594,  642,  369, 1585,  188,  133,  369, 2012,    7,  218,
        412, 1428,   73,    0,    0,    0,    0,    0,    0])

In [56]:
Sequences

[[171, 1, 3808, 942, 336, 311, 211, 2286, 509, 2323],
 [5102, 594, 642, 369, 1585, 188, 133, 369, 2012, 7, 218, 412, 1428, 73],
 [6661, 77, 61, 3017, 141, 706, 6662, 317, 235, 218, 122, 835, 2930, 279],
 [515, 22, 26, 248, 191, 334, 495, 299, 329, 239, 2684],
 [8375, 285, 8376, 369, 330, 604],
 [140, 7082, 4376, 4641, 36, 89, 14, 386, 157, 2205, 1495, 36, 81, 551],
 [4590, 521, 4591, 1564, 337, 1458, 14],
 [9145, 701, 381, 2939, 724, 36, 8, 5429, 807, 978, 979, 9145, 248, 34],
 [15, 133, 13, 302, 1264, 448, 1280, 2056, 379, 18, 3898, 1374],
 [1,
  1033,
  780,
  2791,
  2453,
  4242,
  669,
  2643,
  4222,
  109,
  74,
  13,
  206,
  2733,
  272],
 [471, 35, 43, 93, 1666, 84, 837, 109, 1565, 958, 1750, 109, 1565],
 [7074, 34, 485, 82, 227, 13, 53, 7075, 501, 502, 232],
 [162, 130, 85, 432, 199, 37, 332, 17, 111, 49],
 [3592, 125, 3593, 928, 23, 454, 17, 1313],
 [706, 433, 2176, 17, 241, 4748, 31, 2994],
 [6917, 295, 278, 6918, 6919, 202, 6920, 2, 6921, 87, 743, 893, 6922, 1614],
 [107,

In [57]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize


-------------------------------------------------------------------------------------------------------

In [58]:
#Create Embedding_index
embedding_index ={}
with open('glove.txt',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        coefs=np.asarray(values[1:],dtype="float32")
        embedding_index[word]=coefs



In [59]:
len(embedding_index['the'])

100

In [60]:
type(coefs)

numpy.ndarray

In [61]:
vocab_size

9275

In [62]:
n=2560

In [63]:
word_index.items()

dict_items([('says', 1), ('percent', 2), ('state', 3), ('obama', 4), ('tax', 5), ('us', 6), ('president', 7), ('year', 8), ('people', 9), ('would', 10), ('states', 11), ('one', 12), ('million', 13), ('years', 14), ('jobs', 15), ('voted', 16), ('government', 17), ('new', 18), ('texas', 19), ('federal', 20), ('bill', 21), ('health', 22), ('billion', 23), ('law', 24), ('every', 25), ('care', 26), ('pay', 27), ('taxes', 28), ('wisconsin', 29), ('barack', 30), ('said', 31), ('country', 32), ('united', 33), ('since', 34), ('first', 35), ('last', 36), ('rate', 37), ('women', 38), ('get', 39), ('money', 40), ('1', 41), ('scott', 42), ('time', 43), ('budget', 44), ('city', 45), ('security', 46), ('even', 47), ('public', 48), ('cut', 49), ('school', 50), ('obamacare', 51), ('florida', 52), ('americans', 53), ('medicare', 54), ('average', 55), ('house', 56), ('gov', 57), ('spending', 58), ('office', 59), ('republican', 60), ('clinton', 61), ('dont', 62), ('american', 63), ('dollars', 64), ('senat

In [64]:
embedding_matrix=np.zeros((2560,20))

In [65]:
embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [66]:
embedding_index.get('without')

array([ 7.1347e-02,  1.3955e-02,  4.2260e-01, -2.7192e-01,  2.4444e-02,
        5.4742e-01, -3.6910e-01,  1.0744e-01,  4.4933e-01, -1.0871e-01,
        3.2773e-01,  2.5514e-02,  1.8760e-01,  7.1466e-02,  4.2016e-01,
       -9.3691e-01, -1.8025e-01,  1.9189e-01, -1.6659e-01, -2.1146e-02,
       -2.6291e-01,  3.9201e-01, -3.0405e-02, -2.3321e-01,  3.1795e-01,
        2.0729e-01, -5.0425e-01, -9.8723e-01,  2.2798e-01, -1.2164e-01,
        3.9037e-01, -3.7607e-02, -6.4537e-01, -5.3506e-01,  1.7855e-01,
       -3.3326e-01, -1.8963e-01, -2.7955e-02, -7.1821e-01, -9.1901e-02,
       -2.1918e-01,  1.5901e-01,  5.5907e-01, -1.8219e-01, -4.6256e-02,
       -4.2019e-01,  2.2868e-01, -4.9949e-01, -2.8316e-01, -9.7350e-01,
        5.9732e-01,  2.6562e-02, -3.1037e-01,  1.3910e+00,  1.2275e-01,
       -2.4381e+00,  2.9824e-02, -3.9304e-01,  1.8912e+00,  3.6262e-01,
       -3.8255e-01,  6.0898e-01, -4.8231e-01,  4.0125e-02,  9.5468e-01,
        1.8180e-03,  6.8199e-01,  8.7480e-02, -1.0827e-01, -2.06

In [67]:
padded_seq[0]

array([ 171,    1, 3808,  942,  336,  311,  211, 2286,  509, 2323,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

---------------------------------------------------------------------------------------------------------------------------------

In [68]:
def get_key(d,value):
    for key, val in d.items():
        if value==val:
            return key

In [69]:
embedding_matrix = np.zeros((2560,20,100))

for i in range(len(padded_seq)):
    for j in range(len(padded_seq[i])):
        try: 
            word = get_key(word_index, padded_seq[i][j])
            vector = embedding_index.get(word)
            embedding_matrix[i][j] = vector
        except:
            embedding_matrix[i][j] = 0

In [70]:
embedding_matrix[0]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [71]:
len(embedding_matrix)

2560

In [72]:
#Create Embedding_matrix
#embedding_matrix=np.zeros((2560,20))
#for word,i in word_index.items():
    #embedding_vector = embedding_index.get(word)
    #if embedding_vector is not None:
        #embedding_matrix[i]=embedding_vector

In [73]:
Sequences[0][0]

171

In [74]:
len(embedding_matrix)

2560

In [75]:
padded_seq[0]

array([ 171,    1, 3808,  942,  336,  311,  211, 2286,  509, 2323,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [76]:
len(np.unique(y))


6

---------------------------------------------------------------------------------------------------------------------------------

In [77]:
from keras.layers import LSTM, Dropout,Dense,Embedding
from keras import Sequential

In [78]:
model = Sequential([
        keras.layers.Embedding(vocab_size,128,
                              ),
        keras.layers.BatchNormalization(),
#         keras.layers.Bidirectional(keras.layers.LSTM(128,return_sequences=True)),
        keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.002)),
        keras.layers.GlobalMaxPool1D(), # Remove flatten layer
        keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.002)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.002)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1,activation='softmax')
    ])


model=Sequential([Embedding(vocab_size+1,128,weights=[embedding_matrix],trainable=False),
                            Dropout(0.2),
                            LSTM(128,return_sequences=True),
                            LSTM(128),
                 Dropout(0.3),
                 Dense(128),
                 Dropout(0.3),
                 Dense(64,),
                 Dense(6,activation='relu')]) 

In [79]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1187200   
                                                                 
 batch_normalization (BatchN  (None, None, 128)        512       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, None, 128)         16512     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0

In [80]:
history =model.fit(embedding_matrix, 
                        y_train,
                        batch_size=512,
                        epochs=20,
                        verbose=1,
                        validation_split=0.2)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\javee\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\javee\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\javee\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\javee\anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\javee\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\javee\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 214, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la

In [ ]:
y_preds =model.predict(x_test, batch_size=256)